# Project Overview

In this project, we will create a simple classifier that determines: CHIHUAHUA… or MUFFIN!

First, let's see what we will cover in this tutorial!

* Review of Machine Learning pipeline, Deep Learning and neurual networks
* Install packages and set up the environment
* Start Deep Learning 
    * Load the dataset
    * Define train parameters
    * Create train loop (building our model)
    * Save the model
* Test the result

# Machine Learning pipeline

In general, Machine Learning means that we let the computer "learn" from some existing dataset，or extract the "pattern" into a model. Later the computer could use the model to predict new data that the it has never seen before.

<center><img src="resources/dl_pipeline.png" width="700"></center>

## Machine Learning vs Deep Learning

In short, Deep Learning is as subset of Machine Learning, but can be a really powerful tool to use.

<br>
<center><img src="resources/ml_vs_dl.png" width ="600"></center>
<br>

[Image source](https://www.onepetro.org/conference-paper/URTEC-2901881-MS)

An example of deep neural networks that might not be "deep" enough (only two hidden layers).

<br>
<center><img src="resources/neural_network_layers.jpg" width ="600" align="center"/></center>

[Image source]( http://cs231n.github.io/neural-networks-1/)

In this project, we are going to use an exisiting model (ResNet) that is trained by someone else and build our own model above it.

<center><img src="resources/folders.png" width="800"></center>

# == Part 0a: Installation and Setup ==

##### Install Python

https://www.python.org/downloads/release/python-373/

##### Install prerequisite dependencies
```
pip install numpy jupyter matplotlib pillow tqdm
```

##### Install PyTorch! https://pytorch.org/get-started/locally/

<img src="Pytorch_logo.png" width="300">
<img src="install_pytorch_mac_cpu.png" width="800">

##### Open this interactive Notebook on your computer!

```bash
git clone https://github.com/sjsumlclub/[OUR_WORKSHOP].git
cd [OUR_WORKSHOP]
jupyter notebook
```

# == Part 0b: Download Data ==
https://www.kaggle.com/pmigdal/alien-vs-predator-images

# == Part 1: Deep Learning ==

Credit for the original code base goes to https://deepsense.ai/keras-vs-pytorch-avp-transfer-learning/! We've modified it to fit this workshop.

##### Generic Python imports

In [0]:
import numpy as np               # linear algebra library
import matplotlib.pyplot as plt  # graphing library
from PIL import Image            # (pillow): image manipulation and loading
%matplotlib inline               # special Jupyter notebook command to show graphs in this notebook


##### Deep learning imports

In [0]:
import torch                                            # PyTorch deep learning framework
from torchvision import datasets, models, transforms    # extension to PyTorch for dataset management
import torch.nn as nn                                   # neural networks module of PyTorch, to let us define neural network layers
from torch.nn import functional as F                    # special functions
import torch.optim as optim                             # optimizers


# Data Loading


#### @AJ_Comment TODO: what if we take out normalization and data augmentation at first, so that they can try adding it themselves later?
#### Distribution of RGB channels on ImageNet; we normalize the channels for more consistent training

In [0]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

In [0]:
# a dictionary of data transforms for our train and validation set
train_transforms = transforms.Compose([
    # resize to resnet input size
    transforms.Resize((224,224)),
    # do some data augmentation
    transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
    transforms.RandomHorizontalFlip(),
    # transform image to PyTorch tensor object
    transforms.ToTensor(),
    normalize
])

validation_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])

In [0]:
image_datasets = {
    'train':
        datasets.ImageFolder('data/train',train_transforms),
    'validation':
        datasets.ImageFolder('data/validation', validation_transforms)}
 
dataloaders = {
    'train':
        torch.utils.data.DataLoader(
            image_datasets['train'],
            batch_size=32,
            shuffle=True,
            num_workers=4),
    'validation':
        torch.utils.data.DataLoader(
            image_datasets['validation'],
            batch_size=32,
            shuffle=False,
            num_workers=4)}


# Define Train Parameters

#### Choose GPU (cuda) or CPU for training

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # cuda:0 means the first cuda device found

#### Choose our neural network architecture

In [0]:
model = models.resnet18(pretrained=True).to(device)                      # load our simple neural network

#### We freeze the internal layers so they don't train them on our new dataset. Instead, we add a couple layers at the end that will do the learning for our dataset.

In [0]:
# Freeze the internal layers
for param in model.parameters():
    param.requires_grad = False

In [0]:
last_layer_size = model.fc.in_features

In [0]:
# Add two layers at the end
model.fc = nn.Sequential(
    nn.Linear(layer_layer_size, 128),           # layer 1
    nn.ReLU(inplace=True),          # activate layer 1
    nn.Linear(128, 2)).to(device)   # layer 2

#### Define the loss function and optimizer for training

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())


# Create our train loop

#### Define the train loop

In [0]:
from tqdm import tnrange, tqdm_notebook # import sexy progress bars

def train_model(model, criterion, optimizer, num_epochs=3):
    # train for x epochs. an epoch is a full iteration through our dataset
    for epoch in tnrange(num_epochs, desc="Total progress", unit="epoch"):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # first train one step and update weights; then after that train step, check our validation loss
        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm_notebook(dataloaders[phase], desc=phase, unit="batch", leave=False):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])
            print(f'{phase} loss: {epoch_loss:.4f}, acc: {epoch_acc:.4f}')

        print()

    return model

#### run the train function

In [0]:
model_trained = train_model(model, criterion, optimizer, num_epochs=3)


# Save the model to our computer

In [0]:
import os

MODEL_DIR = "models"
os.makedirs(MODEL_DIR, exist_ok=True)

In [0]:
model_file = os.path.join(MODEL_DIR, "weights.pth")
torch.save(model_trained.state_dict(),model_file)

In [0]:
model = models.resnet18(pretrained=False).to(device)
model.fc = nn.Sequential(
    nn.Linear(last_layer_size, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 2)).to(device)
model.load_state_dict(torch.load(model_file))


# Visually test model performance

Choose some images

In [0]:
validation_img_paths = ["data/validation/alien/11.jpg",
                        "data/validation/alien/22.jpg",
                        "data/validation/predator/33.jpg"]
img_list = [Image.open(img_path) for img_path in validation_img_paths]


Calculate the prediction probabilities

In [0]:
validation_batch = torch.stack( [validation_transforms(img).to(device) for img in img_list] )
 
pred_logits_tensor = model(validation_batch)
pred_probs = F.softmax(pred_logits_tensor, dim=1).cpu().data.numpy()


Plot the images

In [0]:
fig, axs = plt.subplots(1, len(img_list), figsize=(20, 5))
for i, img in enumerate(img_list):
    ax = axs[i]
    ax.axis('off')
    ax.set_title("{:.0f}% Alien, {:.0f}% Predator".format(100*pred_probs[i,0],
                                                          100*pred_probs[i,1]))
    ax.imshow(img)
